In [1]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# self define
sys.path.append('../../utils/')
import Prepare
import Overview
import Completing
import Correcting
import Modelling

In [2]:
item_cat = pd.read_csv('../data/item_categories.csv')
items = pd.read_csv('../data/items.csv')
shops = pd.read_csv('../data/shops.csv')
sales_train = pd.read_csv('../data/sales_train.csv')
sales_test = pd.read_csv('../data/test.csv.gz')

In [3]:
extract_train = sales_train.merge(sales_test[['shop_id']].drop_duplicates(), how = 'inner')

In [4]:
outl = Correcting.detect_outliers_with_IQR_groupby(extract_train, 0, ['item_price'], 'item_id')

100%|██████████| 21005/21005 [01:34<00:00, 222.59it/s]


In [20]:
extract_train.loc[outl].sort_values('date_block_num').head(20)

TypeError: cannot use label indexing with a null key

In [5]:
tmp = extract_train.groupby('item_id').describe()

In [12]:
tmp

date_block_num                                                        \
                 count       mean        std   min    25%   50%    75%   max   
item_id                                                                        
1                  6.0  18.000000   2.529822  15.0  15.75  18.5  19.75  21.0   
16                 1.0  20.000000        NaN  20.0  20.00  20.0  20.00  20.0   
17                 1.0  20.000000        NaN  20.0  20.00  20.0  20.00  20.0   
18                 1.0  19.000000        NaN  19.0  19.00  19.0  19.00  19.0   
19                 1.0   0.000000        NaN   0.0   0.00   0.0   0.00   0.0   
26                 1.0  21.000000        NaN  21.0  21.00  21.0  21.00  21.0   
27                36.0   6.638889   5.457120   0.0   1.75   6.0  10.25  17.0   
28                67.0   5.432836   3.614885   0.0   2.00   6.0   8.50  14.0   
29                13.0   5.076923   4.838415   0.0   0.00   6.0   9.00  14.0   
30              1078.0   6.775510   7.466277   1.0   2.00   3.0  11.00  33.0   
31               876.0  10.031963  10.623161   1.0   1.00   4.0  18.00  33.0   
32              1554.0  10.848777   9.664143   0.0   2.00   8.0  17.00  33.0   
33               693.0  13.760462   9.765919   0.0   6.00  12.0  22.00  33.0   
34               101.0   5.445545   3.766896   0.0   3.00   5.0   7.00  16.0   
35               120.0   0.950000   1.873522   0.0   0.00   1.0   1.00  20.0   
36                72.0   9.180556   6.818216   1.0   3.00   8.5  14.00  26.0   
37               114.0   7.903509   4.965312   2.0   3.25   7.0  10.00  23.0   
38                54.0  25.333333   4.278070  18.0  22.00  24.0  30.00  32.0   
39                 5.0  18.000000   9.380832  11.0  11.00  12.0  25.00  31.0   
40                82.0  10.036585   7.371023   0.0   4.00   8.0  14.00  33.0   
41                31.0   3.548387   3.064188   0.0   1.50   3.0   5.00  12.0   
42                90.0  12.488889   8.150611   0.0   6.00  11.0  17.00  33.0   
43                 7.0   4.142857   2.794553   0.0   2.00   5.0   6.50   7.0   
44                85.0  10.470588   6.582061   0.0   4.00  11.0  15.00  24.0   
45               130.0  15.184615   9.668766   0.0   6.00  16.0  23.00  33.0   
46                35.0   6.714286   5.067610   0.0   3.00   6.0  10.00  23.0   
47                46.0   5.000000   4.798148   0.0   2.00   4.0   7.00  21.0   
48                45.0   6.644444   5.330681   0.0   3.00   5.0  11.00  18.0   
49               151.0  13.860927   9.676115   0.0   5.00  13.0  21.00  33.0   
50                 1.0   0.000000        NaN   0.0   0.00   0.0   0.00   0.0   
...                ...        ...        ...   ...    ...   ...    ...   ...   
22140            244.0  18.729508   8.887071   0.0  16.00  21.0  25.00  31.0   
22141             30.0  14.000000   0.742781  13.0  14.00  14.0  14.00  16.0   
22142              7.0  14.142857   0.899735  13.0  14.00  14.0  14.00  16.0   
22143            507.0  15.122288   1.864171  14.0  14.00  14.0  15.00  22.0   
22144             90.0  15.055556   2.362445  14.0  14.00  14.0  15.00  32.0   
22145            224.0  15.919643   4.530194  14.0  14.00  14.0  15.00  33.0   
22146             22.0  18.772727   6.255906   2.0  15.00  18.5  21.75  31.0   
22147             48.0   4.375000   3.330229   0.0   2.00   4.0   6.25  15.0   
22148              2.0   5.500000   0.707107   5.0   5.25   5.5   5.75   6.0   
22149             26.0   5.500000   3.190611   0.0   3.25   6.0   7.75  11.0   
22150             45.0   8.600000   3.927757   1.0   6.00   7.0  11.00  18.0   
22151            185.0   0.621622   1.072161   0.0   0.00   0.0   1.00   6.0   
22152             70.0   4.657143   2.314953   2.0   3.00   4.0   6.00  13.0   
22153              9.0  21.111111   4.621808  16.0  19.00  20.0  23.00  31.0   
22154             48.0  15.500000  14.945290   0.0   0.00   9.0  31.00  33.0   
22155             55.0   8.218182   6.349952   3.0   4.00   5.0  10.50  32.0   
22156   

In [27]:
a = extract_train.groupby('item_id')

In [29]:
a[['item_price', 'shop_id']]

In [26]:
b

,item_id,count,mean,std,min,25%,50%,75%,max
0,1,6.0,18.000000,2.529822,15.0,15.75,18.5,19.75,21.0
1,16,1.0,20.000000,NaN,20.0,20.00,20.0,20.00,20.0
2,17,1.0,20.000000,NaN,20.0,20.00,20.0,20.00,20.0
3,18,1.0,19.000000,NaN,19.0,19.00,19.0,19.00,19.0
4,19,1.0,0.000000,NaN,0.0,0.00,0.0,0.00,0.0
5,26,1.0,21.000000,NaN,21.0,21.00,21.0,21.00,21.0
6,27,36.0,6.638889,5.457120,0.0,1.75,6.0,10.25,17.0
7,28,67.0,5.432836,3.614885,0.0,2.00,6.0,8.50,14.0
8,29,13.0,5.076923,4.838415,0.0,0.00,6.0,9.00,14.0
9,30,1078.0,6.775510,7.466277,1.0,2.00,3.0,11.00,33.0
